# I025

# Dhup Mehta

# GenAI Lab 10

# B. Tech AI | Sem- 7

# Batch 1

In [43]:
!pip install langgraph==0.3.2 mistralai pandas langchain-mistralai langchain-core -q

In [44]:
import getpass
import os

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")
api_key = os.environ["MISTRAL_API_KEY"]

In [45]:
import json
import pandas as pd
from langgraph.graph import StateGraph, END
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import ToolMessage

In [55]:
movie_data = {
    'movie_id': ['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20'],
    'title': [
        'Inception', 'The Dark Knight', 'Pulp Fiction', 'Forrest Gump', 'The Matrix',
        'Avatar', 'Titanic', 'Jurassic Park', 'The Avengers', 'Interstellar',
        'Parasite', 'The Lion King', 'The Shawshank Redemption', 'The Godfather',
        'Fight Club', 'Gladiator', 'Saving Private Ryan', 'Spirited Away',
        'The Silence of the Lambs', 'Whiplash'
    ],
    'genre': [
        'Sci-Fi', 'Action', 'Crime', 'Drama', 'Sci-Fi', 'Sci-Fi', 'Romance',
        'Sci-Fi', 'Action', 'Sci-Fi', 'Thriller', 'Animation', 'Drama', 'Crime',
        'Drama', 'Action', 'War', 'Animation', 'Thriller', 'Drama'
    ],
    'release_year': [
        2010, 2008, 1994, 1994, 1999, 2009, 1997, 1993, 2012, 2014, 2019,
        1994, 1994, 1972, 1999, 2000, 1998, 2001, 1991, 2014
    ],
    'director': [
        'Christopher Nolan', 'Christopher Nolan', 'Quentin Tarantino',
        'Robert Zemeckis', 'Wachowskis', 'James Cameron', 'James Cameron',
        'Steven Spielberg', 'Joss Whedon', 'Christopher Nolan', 'Bong Joon Ho',
        'Roger Allers', 'Frank Darabont', 'Francis Ford Coppola', 'David Fincher',
        'Ridley Scott', 'Steven Spielberg', 'Hayao Miyazaki', 'Jonathan Demme', 'Damien Chazelle'
    ],
    'imdb_rating': [
        8.8, 9.0, 8.9, 8.8, 8.7, 7.8, 7.8, 8.1, 8.0, 8.6, 8.6, 8.5, 9.3,
        9.2, 8.8, 8.5, 8.6, 8.6, 8.6, 8.5
    ],
    'box_office_revenue_usd': [
        829900000, 1004900000, 213900000, 678200000, 465300000, 2923700000,
        2257900000, 1109800000, 1518800000, 701800000, 258700000, 968500000,
        58500000, 291000000, 101200000, 465400000, 482300000, 395800000,
        272700000, 49000000
    ]
}

df_movies = pd.DataFrame(movie_data)
print(df_movies)

   movie_id                     title      genre  release_year  \
0       M01                 Inception     Sci-Fi          2010   
1       M02           The Dark Knight     Action          2008   
2       M03              Pulp Fiction      Crime          1994   
3       M04              Forrest Gump      Drama          1994   
4       M05                The Matrix     Sci-Fi          1999   
5       M06                    Avatar     Sci-Fi          2009   
6       M07                   Titanic    Romance          1997   
7       M08             Jurassic Park     Sci-Fi          1993   
8       M09              The Avengers     Action          2012   
9       M10              Interstellar     Sci-Fi          2014   
10      M11                  Parasite   Thriller          2019   
11      M12             The Lion King  Animation          1994   
12      M13  The Shawshank Redemption      Drama          1994   
13      M14             The Godfather      Crime          1972   
14      M1

In [51]:
def get_movie_details(df: pd.DataFrame, title: str) -> str:
    movie_info = df[df['title'].str.lower() == title.lower()]
    if not movie_info.empty: return json.dumps(movie_info.to_dict(orient='records')[0])
    return json.dumps({'error': f'Movie with title "{title}" not found.'})

def find_movies_by_director(df: pd.DataFrame, director: str) -> str:
    movies = df[df['director'].str.lower() == director.lower()]['title'].tolist()
    if movies: return json.dumps({'movies': movies})
    return json.dumps({'error': f'No movies found for director "{director}".'})

def get_highest_rated_movie_by_genre(df: pd.DataFrame, genre: str) -> str:
    genre_movies = df[df['genre'].str.lower() == genre.lower()]
    if not genre_movies.empty:
        highest_rated = genre_movies.loc[genre_movies['imdb_rating'].idxmax()]
        return json.dumps({'title': highest_rated['title'], 'imdb_rating': highest_rated['imdb_rating']})
    return json.dumps({'error': f'No movies found for genre "{genre}".'})

def compare_box_office(df: pd.DataFrame, movie_title_1: str, movie_title_2: str) -> str:
    movie1 = df[df['title'].str.lower() == movie_title_1.lower()]
    movie2 = df[df['title'].str.lower() == movie_title_2.lower()]
    if movie1.empty: return json.dumps({'error': f'Movie "{movie_title_1}" not found.'})
    if movie2.empty: return json.dumps({'error': f'Movie "{movie_title_2}" not found.'})
    revenue1 = movie1['box_office_revenue_usd'].iloc[0]
    revenue2 = movie2['box_office_revenue_usd'].iloc[0]
    if revenue1 > revenue2:
        return json.dumps({'higher_grossing_movie': movie_title_1, 'revenue_difference_usd': int(revenue1 - revenue2)})
    return json.dumps({'higher_grossing_movie': movie_title_2, 'revenue_difference_usd': int(revenue2 - revenue1)})


In [53]:
tools = [
    {"type": "function", "function": {"name": "get_movie_details", "description": "Get all available details for a specific movie by its title.", "parameters": {"type": "object", "properties": {"title": {"type": "string", "description": "The title of the movie."}}, "required": ["title"]}}},
    {"type": "function", "function": {"name": "find_movies_by_director", "description": "List all movies in the dataset directed by a specific director.", "parameters": {"type": "object", "properties": {"director": {"type": "string", "description": "The name of the director."}}, "required": ["director"]}}},
    {"type": "function", "function": {"name": "get_highest_rated_movie_by_genre", "description": "Find the movie with the highest IMDb rating within a specific genre.", "parameters": {"type": "object", "properties": {"genre": {"type": "string", "description": "The genre to search within (e.g., Sci-Fi, Action)."}}, "required": ["genre"]}}},
    {"type": "function", "function": {"name": "compare_box_office", "description": "Compare the box office revenue of two movies.", "parameters": {"type": "object", "properties": {"movie_title_1": {"type": "string", "description": "The title of the first movie."}, "movie_title_2": {"type": "string", "description": "The title of the second movie."}}, "required": ["movie_title_1", "movie_title_2"]}}}
]
model = ChatMistralAI(model="mistral-large-latest", api_key=api_key)
State = dict

def planner(state: State) -> State:
    messages = state.get("messages", [])
    response = model.invoke(messages, tool_choice="auto", tools=tools)
    state["messages"].append(response)

    state["tool_calls"] = response.tool_calls
    return state

names_to_functions = {
    "get_movie_details": get_movie_details,
    "find_movies_by_director": find_movies_by_director,
    "get_highest_rated_movie_by_genre": get_highest_rated_movie_by_genre,
    "compare_box_office": compare_box_office,
}

def execute_tools(state: State) -> State:
    tool_calls = state.get("tool_calls", [])
    messages = state.get("messages", [])
    tool_results = []
    for tool_call in tool_calls:
        function_name = tool_call['name']
        function_params = tool_call['args']
        if function_name in names_to_functions:
            function_result = names_to_functions[function_name](df_movies, **function_params)
            tool_results.append(ToolMessage(content=function_result, tool_call_id=tool_call['id']))
    messages.extend(tool_results)
    state["messages"] = messages
    state["tool_calls"] = None
    return state


def check_tool_needed(state: State) -> str:

    if state["messages"][-1].tool_calls:
        return "call_tool"
    else:
        return "__end__"

graph_builder = StateGraph(State)
graph_builder.set_entry_point("planner")

graph_builder.add_node("planner", planner)
graph_builder.add_node("call_tool", execute_tools)

graph_builder.add_conditional_edges(
    "planner",
    check_tool_needed,
    {
        "call_tool": "call_tool",
        "__end__": END,
    },
)

graph_builder.add_edge("call_tool", "planner")

graph = graph_builder.compile()
print("\nLangGraph agent for movies compiled successfully!")



LangGraph agent for movies compiled successfully!


In [49]:
def run_query(user_query: str):
    messages = [
        {"role": "system", "content": "You are a movie information assistant. Answer the user's questions concisely based on the provided data."},
        {"role": "user", "content": user_query}
    ]
    state = {"messages": messages}
    result = graph.invoke(state)
    return result["messages"][-1].content

print("\nRunning Queries")
#query1 = "What movies did Christopher Nolan direct?"

#query2 = "What is the highest-rated Sci-Fi movie in your data?"

query3 = "Which movie made more money, Avatar or Titanic?"

#query4 = "Tell me the genre and IMDB rating for The Dark Knight."

print(f"User: {query3}")
print(f"Agent: {run_query(query3)}\n")


Running Queries
User: Which movie made more money, Avatar or Titanic?
Agent: **Avatar** made more money than *Titanic* by **$665.8 million USD**.

